<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Figure_3/Figure_3b/1_validate_palmdb2palmdb_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validate the alignment of RdRP nucleotide sequences to the corresponding amino acid references with kallisto translated search - Part 1/2: Alignment

In [ ]:
!pip install -q biopython kb_python
from Bio import SeqIO
import pandas as pd
import numpy as np
from itertools import product
import kb_python.utils as kb_utils
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
from tqdm import tqdm
TQDM_BAR_FORMAT = (
    "{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.9 MB/s eta 0:00:00


Install kallisto and bustools:

In [ ]:
# Install kallisto from source
!git clone -q https://github.com/pachterlab/kallisto.git
!cd kallisto && mkdir build && cd build && cmake .. && make

# Install bustools from source
!git clone -q https://github.com/BUStools/bustools.git
!cd bustools && mkdir build && cd build && cmake .. && make

# Define paths to kallisto and bustools binaries
kallisto = "/content/kallisto/build/src/kallisto"
bustools = "/content/bustools/build/src/bustools"

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_CXX17
-- Performing Test COMPILER_SUPPORTS_CXX17 - Success
shared build
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_P

Load data:

In [ ]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

virus_t2g = "palmdb_clustered_t2g.txt"
u_tax_dp = "ID_to_taxonomy_mapping.csv"
palmdb = "palmdb_rdrp_seqs.fa"

--2024-12-01 04:27:52--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4561689 (4.3M) [text/plain]
Saving to: ‘palmdb_clustered_t2g.txt’

palmdb_clustered_t2 100%[===================>]   4.35M  --.-KB/s    in 0.08s   

2024-12-01 04:27:52 (53.9 MB/s) - ‘palmdb_clustered_t2g.txt’ saved [4561689/4561689]

--2024-12-01 04:27:52--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, 

Reverse translate virus ref amino acid sequences to nucleotides:

In [ ]:
# Adapted from https://github.com/Edinburgh-Genome-Foundry/DnaChisel
from Bio.Data import CodonTable

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_backtranslation_table(table_name="Standard"):
    table = CodonTable.unambiguous_dna_by_name[table_name]
    back_translation_table = {}
    for codon, amino_acid in table.forward_table.items():
        if amino_acid not in back_translation_table:
            back_translation_table[amino_acid] = []
        back_translation_table[amino_acid].append(codon)
    back_translation_table["*"] = table.stop_codons
    back_translation_table["START"] = table.start_codons
    back_translation_table["X"] = list(set(flatten(back_translation_table.values())) - set(back_translation_table["*"]))
    back_translation_table["B"] = back_translation_table["N"] + back_translation_table["D"]
    back_translation_table["J"] = back_translation_table["L"] + back_translation_table["I"]
    back_translation_table["Z"] = back_translation_table["E"] + back_translation_table["Q"]
    return back_translation_table

def reverse_translate(protein_sequence, randomize_codons=False, table="Standard"):
    """Return a DNA sequence which translates to the provided protein sequence.

    Parameters
    ----------

    protein_sequence
      A sequence string of aminoacids, e.g. "MVKK..."

    table
      Genetic code table to use (e.g. 'Standard', 'Bacterial', etc.).
      See dnachisel.biotools.CODON_TABLE_NAMES for a list of available genetic
      code tables.

    randomize_codons
      If False, the first valid codon found is used for each, which can create
      biases (GC content, etc.), if True, each amino acid gets replaced by a
      randomly selected codon for this amino acid.
    """
    backtranslation_table = get_backtranslation_table(table_name=table)
    if randomize_codons:
        random_numbers = np.random.randint(0, 1000, len(protein_sequence))
        random_indices = [
            random_number % len(backtranslation_table[aa])
            for aa, random_number in zip(protein_sequence, random_numbers)
        ]
        return "".join(
            [
                backtranslation_table[aa][random_indice]
                for aa, random_indice in zip(protein_sequence, random_indices)
            ]
        )
    return "".join([backtranslation_table[aa][0] for aa in protein_sequence])

In [ ]:
# Reverse translate virus ref amino acid sequences to nucleotides and save to fasta
palmdb_nn_fasta = "palmdb_rdrp_seqs_nucleotides.fasta"

with open(palmdb) as handle, open(palmdb_nn_fasta, "w") as palm_nuc:
    for record in SeqIO.parse(handle, "fasta"):
        palm_nuc.write(">" + str(record.id) + "\n")
        palm_nuc.write(reverse_translate(str(record.seq), table="Standard") + "\n")

Create fake R1 file with barcodes for rev translated PalmDB:

In [ ]:
# Create a fake barcode for each sequence in the PalmDB
barcodes = [''.join(i) for i in product(["A", "C", "G", "T"], repeat=10)]

print(len(barcodes))
barcodes[:10]

1048576


['AAAAAAAAAA',
 'AAAAAAAAAC',
 'AAAAAAAAAG',
 'AAAAAAAAAT',
 'AAAAAAAACA',
 'AAAAAAAACC',
 'AAAAAAAACG',
 'AAAAAAAACT',
 'AAAAAAAAGA',
 'AAAAAAAAGC']

In [ ]:
# Create fake R1 file with barcodes
virus_ids = []
palmdb_nn_R1 = "palmdb_barcodes_R1.fasta"
with open(palmdb_nn_R1, "w") as fake_R1:
    for (barcode, record) in zip(barcodes[:296561], SeqIO.parse(palmdb_nn_fasta, "fasta")):
        fake_R1.write(">" + record.id + "\n")
        fake_R1.write(barcode + "\n")
        virus_ids.append(record.id)

In [ ]:
len(virus_ids)

296561

In [ ]:
bc2virus = pd.DataFrame()
bc2virus["virus_ID"] = virus_ids
bc2virus["barcode"] = barcodes[:296561]
bc2virus.head()

,virus_ID,barcode
0,u1,AAAAAAAAAA
1,u2,AAAAAAAAAC
2,u3,AAAAAAAAAG
3,u4,AAAAAAAAAT
4,u5,AAAAAAAACA


Create reference index from optimized PalmDB amino acid sequences:

In [ ]:
virus_index = "virus_index.idx"

!$kallisto index \
  --aa \
  -t 2 \
  -i $virus_index \
  $palmdb


[build] loading fasta file palmdb_rdrp_seqs.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508718 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 2040536 unitigs
CompactedDBG::construct(): After split (1/1): 2

Align:

In [ ]:
# Use kallisto to align with fake barcodes
out_folder = "palmdb_mapping_validation"

!$kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -x 0,0,0:0,0,10:1,0,0 \
        -t 2 \
        $palmdb_nn_R1 $palmdb_nn_fasta

!$bustools sort -o $out_folder/output_sorted.bus $out_folder/output.bus

!$bustools count \
    --genecounts \
    --cm \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted.bus


[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,756
[quant] running in paired-end mode
[quant] will process sample 1: palmdb_barcodes_R1.fasta
                               palmdb_rdrp_seqs_nucleotides.fasta
[quant] finding pseudoalignments for the reads ... done
[quant] processed 296,561 reads, 286,589 reads pseudoaligned

 all fits in buffer
Read in 286589 BUS records
reading time 0.003182s
sorting time 0.043599s
writing time 0.031358s


In [ ]:
# Load data
# Filepath to counts
X = f"{out_folder}/bustools_count/output.mtx"
# Filepath to barcode metadata
var_path = f"{out_folder}/bustools_count/output.genes.txt"
# Filepath to gene metadata
obs_path = f"{out_folder}/bustools_count/output.barcodes.txt"

# Create AnnData object
adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)

adata

AnnData object with n_obs × n_vars = 286589 × 99228

In [ ]:
# Add virus IDs
adata.obs = adata.obs.merge(bc2virus, how="left", on="barcode", validate="one_to_one").set_index("virus_ID")
adata.obs

,barcode
virus_ID,
u1,AAAAAAAAAA
u2,AAAAAAAAAC
u3,AAAAAAAAAG
u4,AAAAAAAAAT
u5,AAAAAAAACA
...,...
u296618,CAGACGCGGT
u296619,CAGACGCGTA
u296620,CAGACGCGTC


Plot heatmap of ID mapping:

In [ ]:
# %%time
# fig, ax = plt.subplots(figsize = (20,20))
# im = ax.imshow(adata.X.todense())
# plt.savefig("palmdb2palmdb_heatmap.png", dpi=300, bbox_inches="tight")

### Plot fraction of each taxa assigned correctly:

In [ ]:
tax_df = pd.read_csv(u_tax_dp)
tax_df

,ID,rep_ID,phylum,class,order,family,genus,species,strandedness
0,u1,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...,+ssRNA
1,u10066,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...,+ssRNA
2,u10121,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...,+ssRNA
3,u10124,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...,+ssRNA
4,u102609,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...,+ssRNA
...,...,...,...,...,...,...,...,...,...
296556,u296608,u296608,.,.,.,.,.,.,unknown
296557,u296609,u296609,.,.,.,.,.,.,unknown
296558,u296613,u296613,.,.,.,.,.,.,unknown
296559,u296616,u296616,.,.,.,.,.,.,unknown


In [ ]:
adata.X.sum()

285527.0

Get fractions of correct/incorrect mapping for each ID for each tax:

In [ ]:
def get_acc_fractions(taxon):
    df = pd.DataFrame()
    df["mapping"] = ["correct", "incorrect", "not_aligned", "multimapped"]

    total_n = len(tax_df[taxon].unique())
    n = 1

    for tax_name in tax_df[taxon].unique():
        correct = 0
        incorrect = 0
        not_aligned = 0
        multimapped = 0

        with tqdm(total=len(tax_df[tax_df[taxon]==tax_name]["ID"].values), bar_format=TQDM_BAR_FORMAT) as pbar:
            pbar.set_description(f"Checking IDs for {tax_name} ({n}/{total_n})")

            for virus_id in tax_df[tax_df[taxon]==tax_name]["ID"].values:
                try:
                    mapping = adata.var[np.array(adata[adata.obs.index == virus_id, :].X.todense())[0] > 0]

                    if len(mapping) == 1:
                        mapped_id = mapping.index[0]

                        if mapped_id == virus_id:
                            correct += 1

                        else:
                            if tax_df[tax_df["ID"]==mapped_id][taxon].values[0] == tax_name:
                                correct += 1

                            else:
                                incorrect += 1

                    else:
                        multimapped += 1

                except IndexError:
                    not_aligned += 1

                pbar.update(1)

        df[tax_name] = [correct, incorrect, not_aligned, multimapped]
        n += 1

    df.to_csv(f"{out_folder}/{taxon}_mapping.csv", index=False)

In [ ]:
get_acc_fractions("phylum")

Checking IDs for Pisuviricota (1/10):  87%|████████▋ | 48614/56104 [elapsed: 48:24 remaining: 06:28]

In [ ]:
get_acc_fractions("class")

Checking IDs for Milneviricetes (24/24): 100%|██████████| 3/3 [elapsed: 00:00 remaining: 00:00]


In [ ]:
get_acc_fractions("order")

Checking IDs for Serpentovirales (30/30): 100%|██████████| 1/1 [elapsed: 00:00 remaining: 00:00]


In [ ]:
get_acc_fractions("family")

Checking IDs for Nanhypoviridae (86/86): 100%|██████████| 3/3 [elapsed: 00:00 remaining: 00:00]


In [ ]:
get_acc_fractions("genus")

Checking IDs for Novirhabdovirus (100/299): 100%|██████████| 33/33 [elapsed: 00:01 remaining: 00:00]
<ipython-input-17-6654d1b3b14c>:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tax_name] = [correct, incorrect, not_aligned, multimapped]
Checking IDs for Barnavirus (101/299): 100%|██████████| 8/8 [elapsed: 00:00 remaining: 00:00]
<ipython-input-17-6654d1b3b14c>:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tax_name] = [correct, incorrect, not_aligned, multimapped]
Checking IDs for Nebovirus (102/299): 100%|████████

In [ ]:
get_acc_fractions("species")

Streaming output truncated to the last 5000 lines.
Checking IDs for Rice stripe mosaic cytorhabdovirus (47/1748): 100%|██████████| 1/1 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Barley yellow dwarf virus GAV (48/1748): 100%|██████████| 6/6 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Middelburg virus (52/1748): 100%|██████████| 3/3 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Tobacco vein banding mosaic virus (53/1748): 100%|██████████| 18/18 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Caraparu orthobunyavirus (54/1748): 100%|██████████| 4/4 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Tomato mottle mosaic virus (55/1748): 100%|██████████| 36/36 [elapsed: 00:01 remaining: 00:00]
Checking IDs for Japanese soil-borne wheat mosaic virus (56/1748): 100%|██████████| 4/4 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Beta vulgaris mitovirus 1 (57/1748): 100%|██████████| 7/7 [elapsed: 00:00 remaining: 00:00]
Checking IDs for Castlerea virus (58/1748): 100%